## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
sys.path.append(os.path.abspath(os.path.join('../../..')))
from apikeys import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Portland,US,45.5234,-122.6762,77.63,57,75,8.05,broken clouds
1,1,Busselton,AU,-33.6500,115.3333,59.58,80,67,20.83,broken clouds
2,2,Havre-Saint-Pierre,CA,50.2334,-63.5986,66.56,88,75,5.75,broken clouds
3,3,Ayagoz,KZ,47.9714,80.4392,65.35,36,100,11.52,overcast clouds
4,4,Mataura,NZ,-46.1927,168.8643,33.48,96,15,2.91,few clouds
...,...,...,...,...,...,...,...,...,...,...
664,664,Gejiu,CN,23.3060,103.1635,55.99,94,64,3.09,broken clouds
665,665,Zaozerne,UA,45.1582,33.2787,73.33,68,16,13.65,few clouds
666,666,Gasa,PS,31.5000,34.4667,72.88,70,1,4.34,clear sky
667,667,Novska,HR,45.3406,16.9769,72.18,44,2,1.77,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Portland,US,45.5234,-122.6762,77.63,57,75,8.05,broken clouds
10,10,Koroni,GR,36.8000,21.9500,82.08,64,0,5.08,clear sky
17,17,Puerto Ayora,EC,-0.7393,-90.3518,80.56,85,49,5.99,scattered clouds
21,21,Kahului,US,20.8947,-156.4700,87.82,59,20,19.57,few clouds
23,23,Alofi,NU,-19.0595,-169.9187,78.69,78,100,5.75,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
651,651,Gat,IL,31.6100,34.7642,79.48,100,1,1.97,clear sky
656,656,Awbari,LY,26.5921,12.7805,82.08,19,0,7.31,clear sky
657,657,Marsh Harbour,BS,26.5412,-77.0636,83.86,72,0,13.78,clear sky
660,660,Cabangahan,PH,9.0781,122.9469,78.58,80,82,0.78,light rain


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                218
City                   218
Country                218
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

In [37]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Portland,US,77.63,broken clouds,45.5234,-122.6762,
10,Koroni,GR,82.08,clear sky,36.8000,21.9500,
17,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,
21,Kahului,US,87.82,few clouds,20.8947,-156.4700,
23,Alofi,NU,78.69,overcast clouds,-19.0595,-169.9187,
...,...,...,...,...,...,...,...
651,Gat,IL,79.48,clear sky,31.6100,34.7642,
656,Awbari,LY,82.08,clear sky,26.5921,12.7805,
657,Marsh Harbour,BS,83.86,clear sky,26.5412,-77.0636,
660,Cabangahan,PH,78.58,light rain,9.0781,122.9469,


In [38]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

print("Beginning hotel search.")

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping.")
print("Hotel search complete.")

Beginning hotel search.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel search complete.


In [39]:
# 7. Drop the rows where there is no Hotel Name.
droplist = hotel_df.loc[(hotel_df["Hotel Name"] == "")]
droplist

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
131,Panguna,PG,75.70,clear sky,-6.3164,155.4848,
149,Lodja,CD,77.52,overcast clouds,-3.4833,23.4333,
156,Capuluan,PH,78.33,scattered clouds,13.8147,122.5181,
338,Tari,NG,82.44,overcast clouds,12.4086,4.4914,
403,Nalut,LY,82.38,clear sky,30.3333,10.8500,
426,Poum,NC,77.40,overcast clouds,-20.2333,164.0167,
511,Abu Samrah,SY,76.06,clear sky,35.3029,37.1841,
517,Herat,AF,82.83,clear sky,34.5000,62.0000,
544,Malayal,PH,76.80,moderate rain,7.2080,121.9420,


In [40]:
hotel_df.drop(droplist.index, inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Portland,US,77.63,broken clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
10,Koroni,GR,82.08,clear sky,36.8000,21.9500,Apartments Xenios Zeus 1
17,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
21,Kahului,US,87.82,few clouds,20.8947,-156.4700,Maui Seaside Hotel
23,Alofi,NU,78.69,overcast clouds,-19.0595,-169.9187,Taloa Heights
...,...,...,...,...,...,...,...
651,Gat,IL,79.48,clear sky,31.6100,34.7642,OlusHome
656,Awbari,LY,82.08,clear sky,26.5921,12.7805,فندق أوباري
657,Marsh Harbour,BS,83.86,clear sky,26.5412,-77.0636,Abaco Beach Resort
660,Cabangahan,PH,78.58,light rain,9.0781,122.9469,Centralle District Beach Resort


In [41]:
# 8a. Create the output File (CSV)
output_data_file = "./WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [80]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [55]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [83]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(40.0, 30.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))